### **Google Drive Bağlantısı**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### **Kurulumlar**

In [ ]:
!pip install -q --use-deprecated=legacy-resolver tflite-model-maker
!pip install -q pycocotools

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

### **Veriyi Zipten Çıkarma**


In [ ]:
!unzip "/content/drive/MyDrive/d2.zip" 

### **Model Seçimi**

| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|--------------------|-----------|---------------|----------------------|
| EfficientDet-Lite0 | 4.4       | 37            | 25.69%               |
| EfficientDet-Lite1 | 5.8       | 49            | 30.55%               |
| EfficientDet-Lite2 | 7.2       | 69            | 33.97%               |
| EfficientDet-Lite3 | 11.4      | 116           | 37.70%               |
| EfficientDet-Lite4 | 19.9      | 260           | 41.96%               |


In [ ]:
spec = model_spec.get('efficientdet_lite2')

In [ ]:
train_data = object_detector.DataLoader.from_pascal_voc('/content/d2/images/train', '/content/d2/images/train', ['raspberry', 'multimetre', 'salca'])

In [ ]:
validation_data = object_detector.DataLoader.from_pascal_voc('/content/d2/images/test', '/content/d2/images/test', ['raspberry', 'multimetre', 'salca'])

In [ ]:
train_data

### **Model Eğitimi**

In [ ]:
model = object_detector.create(train_data, model_spec=spec, epochs=100, batch_size=8, train_whole_model=True, validation_data=validation_data)

### **Modelini Test Etme**

In [ ]:
model.evaluate(validation_data)

### **Export Alma ve TFLite Modelini Test Etme**



In [ ]:
model.export(export_dir='model.tflite')

In [ ]:
model.evaluate_tflite('/content/model.tflite/model.tflite', validation_data)

### **Resim Üzerinden Test**




In [ ]:
!pip uninstall opencv-python-headless
!pip install opencv-python-headless==4.1.2.30

In [ ]:
import cv2

from PIL import Image

model_path = '/content/model.tflite/model.tflite'

classes = ['???'] * model.model_spec.config.num_classes
label_map = model.model_spec.config.label_map
for label_id, label_name in label_map.as_dict().items():
  classes[label_id-1] = label_name

COLORS = np.random.randint(0, 255, size=(len(classes), 3), dtype=np.uint8)

def preprocess_image(image_path, input_size):
  img = tf.io.read_file(image_path)
  img = tf.io.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.uint8)
  original_image = img
  resized_img = tf.image.resize(img, input_size)
  resized_img = resized_img[tf.newaxis, :]
  return resized_img, original_image


def set_input_tensor(interpreter, image):
  tensor_index = interpreter.get_input_details()[0]['index']
  input_tensor = interpreter.tensor(tensor_index)()[0]
  input_tensor[:, :] = image


def get_output_tensor(interpreter, index):
  output_details = interpreter.get_output_details()[index]
  tensor = np.squeeze(interpreter.get_tensor(output_details['index']))
  return tensor


def detect_objects(interpreter, image, threshold):
  set_input_tensor(interpreter, image)
  interpreter.invoke()

  scores = get_output_tensor(interpreter, 0)
  boxes = get_output_tensor(interpreter, 1)
  count = int(get_output_tensor(interpreter, 2))
  classes = get_output_tensor(interpreter, 3)

  results = []
  for i in range(count):
    if scores[i] >= threshold:
      result = {
        'bounding_box': boxes[i],
        'class_id': classes[i],
        'score': scores[i]
      }
      results.append(result)
  return results


def run_odt_and_draw_results(image_path, interpreter, threshold=0.5):
  _, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']

  preprocessed_image, original_image = preprocess_image(
      image_path, 
      (input_height, input_width)
    )

  results = detect_objects(interpreter, preprocessed_image, threshold=threshold)

  original_image_np = original_image.numpy().astype(np.uint8)
  for obj in results:
    ymin, xmin, ymax, xmax = obj['bounding_box']
    xmin = int(xmin * original_image_np.shape[1])
    xmax = int(xmax * original_image_np.shape[1])
    ymin = int(ymin * original_image_np.shape[0])
    ymax = int(ymax * original_image_np.shape[0])

    class_id = int(obj['class_id'])

    color = [int(c) for c in COLORS[class_id]]
    cv2.rectangle(original_image_np, (xmin, ymin), (xmax, ymax), color, 2)
    y = ymin - 15 if ymin - 15 > 15 else ymin + 15
    label = "{}: {:.0f}%".format(classes[class_id], obj['score'] * 100)
    cv2.putText(original_image_np, label, (xmin, y),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

  original_uint8 = original_image_np.astype(np.uint8)
  return original_uint8

In [ ]:
INPUT_IMAGE_URL = "/content/drive/MyDrive/v4_data/IMG_20210510_085833139.jpg"
DETECTION_THRESHOLD = 0.5 

interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()
detection_result_image = run_odt_and_draw_results(
    INPUT_IMAGE_URL, 
    interpreter, 
    threshold=DETECTION_THRESHOLD
)
Image.fromarray(detection_result_image)